In [ ]:
import torchvision.transforms.functional as F

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import torch
import math
import os
import time
import json
import logging

In [ ]:
# # imports pytorch
# import torch

# # imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')
zip_ref = zipfile.ZipFile("/content/drive/My Drive/MASTER/data_final.zip", 'r')
zip_ref.extractall("/content/tmp/data")
zip_ref.close()

Mounted at /content/drive/


In [ ]:
from google.colab import files
src = list(files.upload().values())[0]
# uploaduj skripte

Saving learner.py to learner.py
Saving meta.py to meta.py
Saving omniglot.py to omniglot.py
Saving omniglotNShot.py to omniglotNShot.py


In [ ]:
import  torch, os
import  numpy as np
from  omniglotNShot import OmniglotNShot
from meta import Meta
from tqdm import tqdm

In [ ]:
torch.manual_seed(222)
np.random.seed(222)

In [ ]:
epoch=60000
n_way=20
k_spt=5
k_qry=15
imgsz=28
imgc=1
task_num=16
meta_lr=0.1
update_lr=0.1
update_step=5
update_step_test=10

In [ ]:
config = [
        ('conv2d', [64, 1, 3, 3, 2, 0]),
        ('relu', [True]),
        ('bn', [64]),
        ('conv2d', [64, 64, 3, 3, 2, 0]),
        ('relu', [True]),
        ('bn', [64]),
        ('conv2d', [64, 64, 3, 3, 2, 0]),
        ('relu', [True]),
        ('bn', [64]),
        ('conv2d', [64, 64, 2, 2, 1, 0]),
        ('relu', [True]),
        ('bn', [64]),
        ('flatten', []),
        ('linear', [n_way, 64])
        ]

In [ ]:
db_train = OmniglotNShot("/content/tmp/data/data_final/omniglot_resized",
                   batchsz=task_num,
                   n_way=n_way,
                   k_shot=k_spt,
                   k_query=k_qry,
                   imgsz=imgsz)

== Found 129840 items 
== Found 6492 classes
data shape: (6492, 20, 28, 28, 1)
DB: train (4800, 20, 28, 28, 1) test (1692, 20, 28, 28, 1)


In [ ]:
device = torch.device('cuda')

maml = Meta(config, update_lr, meta_lr, n_way, k_spt, k_qry, task_num, update_step, update_step_test).to(device)

tmp = filter(lambda x: x.requires_grad, maml.parameters())
num = sum(map(lambda x: np.prod(x.shape), tmp))
print(maml)
print('Total trainable tensors:', num)

Meta(
  (net): Learner(
    conv2d:(ch_in:1, ch_out:64, k:3x3, stride:2, padding:0)
    relu:(True,)
    bn:(64,)
    conv2d:(ch_in:64, ch_out:64, k:3x3, stride:2, padding:0)
    relu:(True,)
    bn:(64,)
    conv2d:(ch_in:64, ch_out:64, k:3x3, stride:2, padding:0)
    relu:(True,)
    bn:(64,)
    conv2d:(ch_in:64, ch_out:64, k:2x2, stride:1, padding:0)
    relu:(True,)
    bn:(64,)
    flatten:()
    linear:(in:64, out:20)
    
    (vars): ParameterList(
        (0): Parameter containing: [torch.float32 of size 64x1x3x3 (GPU 0)]
        (1): Parameter containing: [torch.float32 of size 64 (GPU 0)]
        (2): Parameter containing: [torch.float32 of size 64 (GPU 0)]
        (3): Parameter containing: [torch.float32 of size 64 (GPU 0)]
        (4): Parameter containing: [torch.float32 of size 64x64x3x3 (GPU 0)]
        (5): Parameter containing: [torch.float32 of size 64 (GPU 0)]
        (6): Parameter containing: [torch.float32 of size 64 (GPU 0)]
        (7): Parameter containing: [

In [ ]:
accs_total = []
for step in tqdm(range(60000)):

    x_spt, y_spt, x_qry, y_qry = db_train.next()
    # x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).to(device), \
    #                              torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).to(device)

    # set traning=True to update running_mean, running_variance, bn_weights, bn_bias
    accs = maml(x_spt.to(device), y_spt.to(device), x_qry.to(device), y_qry.to(device))

    if step % 100 == 0:
        print('step:', step, '\ttraining acc:', accs)

    if step % 500 == 0:
        accs = []
        for _ in range(1000//32):
            # test
            x_spt, y_spt, x_qry, y_qry = db_train.next('test')
            # x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).to(device), \
            #                              torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).to(device)

            # split to single task each time
            for x_spt_one, y_spt_one, x_qry_one, y_qry_one in zip(x_spt, y_spt, x_qry, y_qry):
                test_acc = maml.finetunning(x_spt_one.to(device), y_spt_one.to(device), x_qry_one.to(device), y_qry_one.to(device))
                accs.append( test_acc )
                accs_total.append( test_acc )

        # [b, update_step+1]
        accs = np.array(accs).mean(axis=0).astype(np.float16)
        print('Test acc:', accs)

  0%|          | 0/60000 [00:00<?, ?it/s]

step: 0 	training acc: [0.04916667 0.09270833 0.11395833 0.12458333 0.13354167 0.13625   ]


  0%|          | 1/60000 [00:18<301:10:50, 18.07s/it]

Test acc: [0.04813 0.0651  0.0869  0.11127 0.1385  0.1661  0.193   0.2184  0.2422
 0.2642  0.2834 ]


  0%|          | 101/60000 [00:50<5:23:25,  3.09it/s]

...

100%|█████████▉| 59801/60000 [6:06:19<01:04,  3.07it/s]

step: 59800 	training acc: [0.05       0.05       0.05       0.05       0.16       0.74604167]


100%|█████████▉| 59901/60000 [6:06:52<00:32,  3.06it/s]

step: 59900 	training acc: [0.05       0.05       0.05       0.05041667 0.18083333 0.78166667]


100%|██████████| 60000/60000 [6:07:25<00:00,  2.72it/s]


In [ ]:
        accs_total = []
        accs = []
        for _ in tqdm(range(1692)):
            # test
            x_spt, y_spt, x_qry, y_qry = db_train.next('test')
            # x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).to(device), \
            #                              torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).to(device)

            # split to single task each time
            for x_spt_one, y_spt_one, x_qry_one, y_qry_one in zip(x_spt, y_spt, x_qry, y_qry):
                test_acc = maml.finetunning(x_spt_one.to(device), y_spt_one.to(device), x_qry_one.to(device), y_qry_one.to(device))
                accs.append( test_acc )
                accs_total.append( test_acc )

        # [b, update_step+1]
        accs = np.array(accs).mean(axis=0).astype(np.float16)
        print('Test acc:', accs)

100%|██████████| 1692/1692 [16:22<00:00,  1.72it/s]

Test acc: [0.05   0.05   0.05   0.0503 0.1516 0.683  0.7974 0.831  0.8457 0.853
 0.858 ]


In [ ]:
# [0.1998, 0.8027, 0.85, 0.8643, 0.8716, 0.875,  0.8774, 0.8794, 0.881,  0.8823, 0.8833] # 5w1s
# [0.2001, 0.878,  0.931,  0.9424, 0.9478, 0.9497, 0.9507, 0.9507, 0.951,  0.9517, 0.9517] # 5w5s
# [0.05,    0.06854, 0.05115, 0.0706,  0.2006,  0.5796,  0.654,   0.6816,  0.694, 0.7017,  0.7065] 20w1s
# [0.05   0.05   0.05   0.0503 0.1516 0.683  0.7974 0.831  0.8457 0.853 0.858 ] 20w5s